In [3]:
import json
import io
from configparser import ConfigParser

In [4]:
form_json = ''
# Form Recognizer results generated and saved to json 
with open('./Life_application_example.png.json', 'r') as form_file:
    form_json = json.loads(form_file.read())

In [5]:
form_kvp = []
for kvp in form_json['analyzeResult']['keyValuePairs']:
    if(kvp.get('key') is not None and kvp.get('value') is not None ):
        form_kvp.append((kvp['key']['content'],kvp['value']['content']))
        #print("{} - {}".format(kvp['key']['content'],kvp['value']['content']))

    

# Validate Form Recognizer Outputs with OpenAI

In [6]:
import os
import openai
from configparser import ConfigParser

parser=ConfigParser()
_=parser.read('../config.cfg')
openai.api_type = "azure"
openai.api_base = parser.get('openai_api','api_ep')
openai.api_version = "2022-06-01-preview"
openai.api_key = parser.get('openai_api','api_key')
model =  parser.get('openai_api','api_model')

In [7]:
for e in form_kvp[:10]:
    #print(e)
    prompt = "Validate following text is type of {} with True or False:\n'''{}'''\n".format(e[0], e[1])
    response = openai.Completion.create( engine=model,  prompt=prompt, temperature=.5,  max_tokens=400,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)
    #print('Response:')
    print("{} - {}: {}".format(e[0],e[1],response['choices'][0]['text'].strip()))

Name (First, MI, Last) - Mark B. Cuban: True
S.SN / T - 444-82-6666: True
Address - 91 Richmond St.: True
City - Dallas: False
Zip Code - 75201: True
Former Name - Michael Jackson: False
M - :selected:: False
OF - :unselected:: False
Date of Birth (mm/dd/yyyy) - 1/1/70: True
State of Birth - TX: True


# Correct Form Recognizer Results with OpenAI

In [8]:
import os
import openai
from configparser import ConfigParser

parser=ConfigParser()
_=parser.read('../config.cfg')
openai.api_type = "azure"
openai.api_base = parser.get('openai_api','api_ep')
openai.api_version = "2022-06-01-preview"
openai.api_key = parser.get('openai_api','api_key')
model =  parser.get('openai_api','api_model')

In [9]:
# Corrects the first 10 form key value pairs
for e in form_kvp[:10]:
    #print(e)
    prompt = "Reformat following text to type of {}:\n'''{}'''\n".format(e[0], e[1])
    response = openai.Completion.create( engine=model,  prompt=prompt, temperature=.5,  max_tokens=400,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)
    #print('Response:')
    print("{} - {}: {}".format(e[0],e[1],response['choices'][0]['text'].strip()))

Name (First, MI, Last) - Mark B. Cuban: Mark Cuban
S.SN / T - 444-82-6666: S.SN: 444-82-6666
Address - 91 Richmond St.: 91 Richmond Street
City - Dallas: Dallas
Zip Code - 75201: 75201
Former Name - Michael Jackson: Michael Jackson
M - :selected:: :selected:
OF - :unselected:: :unselected:
Date of Birth (mm/dd/yyyy) - 1/1/70: 01/01/1970
State of Birth - TX: Texas
